# Friction Loss

For gravity flows, friction loss is taken care of by the [shallow water equations](https://en.wikipedia.org/wiki/Shallow_water_equations#Dynamic_wave).



For force mains, friction loss can be calculated using several equations,
* Using the frictioon slope derived from the Manning's equation
* Using the Hazen-Williams equation
* Using the Darcy-Weisbach equation

# Manning's equation

When using the Manning's equation, the friction loss is calculated as the friction slope.

$$ S_f = (\frac{n}{1.486})^2\frac{V^2}{R^\frac{4}{3}} $$

where,
* $S_f$ is the friction slope (head loss per unit length) (ft/ft)
* n= the Manning roughness coefficient (sec/m1/3)
* R = the hydraulic radius of the flow cross-section (ft)
* V = flow velocity (ft/sec).

# Hazen-Williams Equation

$$ S_f = \frac{0.6V^{1.852}}{C^{1.852}R^{1.167}}  $$

where,
* $S_f$ is the friction slope (head loss per unit length) (ft/ft)
* C= Hazen-Williams C-factor coefficient
* R = the hydraulic radius of the flow cross-section (ft)
* V = flow velocity (ft/sec).

# Darcy-Weisbach Equation

$$ S_f = \frac{fV^2}{8gR}$$

where,
* $S_f$ is the friction slope (head loss per unit length) (ft/ft)
* f= dimesionless friction factor
* R = the hydraulic radius of the flow cross-section (ft)
* V = flow velocity (ft/sec).


The Hazen-Williams equation is widely used in the United States due to its simplicity, the C-factors are available for commonly used pipe materials and sizes. The Darcy-Weisbach equation is most theoretically correct, however, getting the right friction coefficient can be more complicated.

In [1]:
import sys
ws = r'C:\Users\Mel.Meng\Documents\GitHub\hhnote\hydraulics\forcemain'
sys.path.append(ws)
from friction import *
from icm import *
import os

Example 1-5/1-4 from WWTP Hydraulics book is shown below. For HW, since it uses a special table, I had to guess the C factor.

In [2]:
f = 0.024
d = 0.5 # ft
q = 0.5 #cfs
l = 1000 #ft
V, R = full_pipe_vr(q, d)
sf = dw_sf(f, V, R)
hl = sf*l
print('DW loss = {:.2f} ft(should be 4.8ft)'.format(hl))
n = 0.013
sf = manning_sf(n, R, V)
hl = sf*l
print('Manning loss = {:.2f} ft(should be 8ft)'.format(hl))
C = 119
sf = hw_sf(C, R, V)
hl = sf*l
print('Manning loss = {:.2f} ft(should be 5.5ft)'.format(hl))


DW loss = 4.83 ft(should be 4.8ft)
Manning loss = 7.90 ft(should be 8ft)
Manning loss = 5.49 ft(should be 5.5ft)


# Example from XPSWMM

[Special Force main](https://help.innovyze.com/display/xps/Special+Conduits#SpecialConduits-PumpDischargePipes(ForceMains)) applies the manning's equation to calculate the friction loss. The firction loss and minor losses were subtracted from the pump curve head to account for the force main losses.

![](images/XPSWMM.png)


In [3]:
# XPSWMM
l = 300 #ft length
d = 1 # ft diameter
n = 0.011
k0 = 1.5 #entrance loss
k1 = 1 #exit loss
k2 = 1.2 # value loss
k3 = 0.5 #bend loss

In [4]:
pump_curve = [(0,100,100,), (2,95,93.901), (14,40,-13.861), (20, 2,-107.921)]
columns = ['Flow(cfs)', 'Ori. head(ft)', 'Minor Loss(ft)', 'Friction Loss(ft)', 'Ori head - Minor(ft)', 'Ori h - Minor-Fric(ft)', 'Check']


data = []
for q, h, ck in pump_curve:
    V, R = full_pipe_vr(q, d)
    sf = manning_sf(n, R, V)
    hl = sf*l
    hm = (k0 + k1 + k2 + k3)*V**2/2/32.2
    hall = hl + hm
    data.append([q, h, hm, hl, h-hm, h - hall, ck])


results = pd.DataFrame(data, columns=columns)

In [5]:
results

,Flow(cfs),Ori. head(ft),Minor Loss(ft),Friction Loss(ft),Ori head - Minor(ft),Ori h - Minor-Fric(ft),Check
0,0,100,0.000000,0.000000,100.000000,100.000000,100.000
1,2,95,0.422906,0.673226,94.577094,93.903868,93.901
2,14,40,20.722385,32.988081,19.277615,-13.710466,-13.861
3,20,2,42.290581,67.322614,-40.290581,-107.613195,-107.921


As shown above, when the force main is full. The results matches the hand calculation very well. However, when the force main is not full, the results are close but noticebly different. This might be due to the SWMM5 engine on calculating the hydraulic radius for a link.

# Equivalent Manning’s n in SWMM5

When the force main is flowing partially full, it will calculate the equivalent manning's n to route using the gravity flow. 

![](images/eq%20manning%20n.png)

```
c = 120
n = 1.067*(1/(11/100))**0.04/120 = 0.0097

```
